In [ ]:
#Install the necessary packages

!pip install yfinance
!pip install matplotlib==3.5.3
!pip install arch

In [ ]:
#Standard packages
import numpy as np
import pandas as pd

#Dates
from datetime import datetime, timedelta

#Finance packages
import yfinance as yf
from arch import arch_model

#Statistics
from scipy.stats import t
from scipy.stats import skew, kurtosis

#Plotting packages
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

from matplotlib import rcParams

rcParams["font.size"] = 20
rcParams["axes.labelsize"] = 30

rcParams["xtick.labelsize"] = 16
rcParams["ytick.labelsize"] = 16

rcParams["figure.figsize"] = (8,6)

# Exercise 1. Analysis of the S&P 500 volatility time-series

**1. Download the data**

In [ ]:
start = datetime(1997, 1, 1)
end = datetime.today()

assets = "^GSPC"

df = yf.download(assets, start=start, end=end, progress=False)

df

**2. Compute the log returns and the moving volatility of log returns**

**Clue:** *The pandas.Series.rolling(window).statistic() method computes the moving version of the chosen statistic over the given window size through all the series.*

**Example**: *df["Col1"].rolling(100).mean()*

In [ ]:
window = 25

df["LogRet"] = #CODE
df["MV"] = #CODE

df

**3. Plot the returns together with the returns +- the moving volatility and the returns +- the std deviation (horizontal line). Plot also the moving volatility alone. What do you observe?**

**Clue**: *Basically I ask you to reproduce the figure I uploaded to UIB digital (contact me if you don't find it)*

In [ ]:
#CODE

# Exercise 2. Volatility forecast with an ARCH model

ARCH stands for autoregressive conditional heteroskedasticity, which basically means that the variance of the underlaying stochastic process changes in time (heteroskedasticity).

An ARCH(p) model is constructed by considering random increments $x$ (usually gaussian) with zero mean and variance $\sigma(t_j)=\sigma_j$ modelled trhough an autoregressive process

\begin{equation}
\sigma_j^2=\alpha_o+\alpha_1 x_{j-1}^2+\dots+\alpha_{p} x_{j-p}^2
\end{equation}

To determine the appropriate number of lags to use (value of $p$) we will use some statistical metrics for model selection: the Akaike information criterion (AIC) and the Bayesian Information Criterion (BIC). Both are estimators of prediction error and thereby relative quality of statistical models for a given set of data. Given a collection of models for the data, AIC and BIC estimate the quality of each model, relative to each of the other models. Thus, they provide a mean for model selection.

The AIC for a given model $\mathcal{M}$ is computed as

\begin{equation}
AIC_{\mathcal{M}} = 2k-2\ln(\hat{L})
\end{equation}

where $k$ is the number of parameters of the model and $\hat{L}$ is the maximized value of the likelihood function for the model.

The BIC for a given model $\mathcal{M}$ is computed as

\begin{equation}
BIC_{\mathcal{M}} = k\ln(n)-2\ln(\hat{L})
\end{equation}

where $k$ is the number of parameters of the model, $n$ is the number of data points used in the fit and $\hat{L}$ is the maximized value of the likelihood function for the model.

As you can see in the formulas above, larger likelihood functions (best-fits) contribute to minimise the criteria, but there is a penalty included by the number of the parameters of the model. This is indeed the point of this criteria, maximise the goodness of the fit while minimising the number of parameters used.

**Take home message:** *smaller values of AIC and BIC correspond to better models.*

**1. Fit an ARCH model**

Chose a particular value of $p$ to fit an ARCH model to the data. Analyze the output carefulluy to understand how the library works.

In [ ]:
#We have to remove the NaNs for the model to work and we rescale the data for better predictions (otherwise a warning shows up)
df_fit = df.dropna() * 100

returns = df_fit['LogRet']

am = arch_model(returns, p=5, o=0, q=0,)

res = am.fit()

print("\n")

res.summary()

**2. Determine the optimal value for $p$**

Fit the model using different values of p (e.g. 1 to 25) and compute their AIC and BIC values to determine the optimal value for $p$.

**Indication:** *Use `am.fit(disp=False)` to deactivate the printing of fit messages*

**Clue:** *The `res.aic` and `res.bic` methods return the AIC and BIC values for the fitted model.*

In [ ]:
#CODE

**3. Forecasts using optimal model**

Use the `forecasts = res.forecast(horizon=1, start=0, reindex=True, align=target)` method to compute the predicted volatility in the sample. You can obtain the forecasted variance as `forecasts.variance`.

Plot the predictions along with the moving volatility previously predicted. Do you think the model is performing well?

*If you are curious on what the arguments of this function are, check it out at https://arch.readthedocs.io/en/latest/*

In [ ]:
#CODE

# Exercise 3. Volatility forecast with GARCH models

To fit a GARCH model you just have to select $p,q\neq0$ in the arch_model(returns, p, o, q) function

**1. Fit a GARCH model**

In [ ]:
am = arch_model(returns, p=1, o=0, q=1)

res = am.fit()

print("\n")

print(res.summary)

**2. Determine the optimal values for $p$ and $q$**

Same as before, but with 2 parameters. Plot the results in a 2D plot (imshow, matshow, pcolormesh, contourf, whatever)

**Note:** *This will take a while as you now have to search for 2 optimal parameters.*

In [ ]:
#CODE

**3. Forecasts**

In [ ]:
#CODE

**4. ARCH or GARCH?**

Compare the best fit models $ARCH(p^*)$ and $GARCH(p^*,q^*)$ to determine which is the overall best one.

In [ ]:
#CODE

# Exercise 4. Simulation of ARCH and GARCH processes

**1. Simulate price dynamics with an ARCH and GARCH model**

We will use the ARCH(11) and GARCH(2,2) models, as we previously found that these were the optimal parameters.

In [ ]:
N_steps = 10**4

# Scale the returns (multiplying by 100) for a propper fit
returns_scaled = df['LogRet'].dropna()*100

# Perform the fit
params_arch = arch_model(returns_scaled, p=11, o=0, q=0).fit(disp=False).params
params_garch = arch_model(returns_scaled, p=2, o=0, q=2).fit(disp=False).params

# Scale back the returns (dividing by 100)
sim_arch = arch_model(None, p=11, o=0, q=0).simulate(params_arch, N_steps) / 100
sim_garch = arch_model(None, p=2, o=0, q=2).simulate(params_garch, N_steps) / 100

S_0 = df.iloc[-1]["Adj Close"]

S_t_arch = #CODE
S_t_garch = #CODE

**2. Plot the simulated returns, volatility and prices**

In [ ]:
#CODE

**3. Plot different realisations of the future simulated prices**

In [ ]:
#CODE

**4. Plot the return distribution of the data (S&P 500), and the ones simulated with the ARCH and GARCH models**

In [ ]:
#CODE